# Advanced Machine Learning - programming assignment 3

**Please fill in:**
* Ernö Groeneweg (4279662)
* Otto Mättas (6324363)
* Vincent de Wit (6970877)

## Reinforcement learning with function approximation
In this assignment, you'll design an agent to complete an episodic task. The agent will be looking at a small part of the UU logo, and will have to decide which of the four compass directions to move in, to find the goal in the center as soon as possible.

The following code defines various aspects of the environment.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras import layers, models, optimizers
from tqdm.notebook import tqdm # For progress bars

# Action space
ACTION_NORTH = 0
ACTION_EAST = 1
ACTION_SOUTH = 2
ACTION_WEST = 3
ACTIONS = [ACTION_NORTH, ACTION_EAST, ACTION_SOUTH, ACTION_WEST]

# Constants defining the environment
GOAL = (300, 276)
AVG_MOVEMENT_SIZE = 24

RADIUS = 72
BOUNDARY_WEST = GOAL[0] - RADIUS
BOUNDARY_EAST = GOAL[0] + RADIUS
BOUNDARY_NORTH = GOAL[1] - RADIUS
BOUNDARY_SOUTH = GOAL[1] + RADIUS

WINDOW_SIZE = 28

TIME_LIMIT = 200
TIMEOUT_REWARD = -10.0

im = plt.imread("UU_logo_EN_CMYK.png")

# Convert to one color channel (using only the red channel), with white background
im = im[:,:,0] * im[:,:,3] + (1.0 - im[:,:,3])

# Get a "camera view" at the position indicated by state
# Use reshape=True to format the output as a data point for the neural network
def get_window(state, reshape=False):
    # When indexing the image as an array, switch the coordinates: im[state[1], state[0]]
    window = im[(state[1]-14):(state[1]+14), (state[0]-14):(state[0]+14)]
    if reshape:
        return np.reshape(window, (1, 28, 28, 1))
    else:
        return window

ModuleNotFoundError: No module named 'keras'

The following three images show:
* The original image, with a red dot marking the goal and a blue rectangle marking the area where the center of agent must remain. A movement that would take the agent outside this rectangle, places him at the boundary instead.
* What the agent sees if he is exactly at the goal.
* The part of the image the agent might see (this is slightly bigger than the blue box, because of the camera's size).

In [ ]:
plt.imshow(im[:,:], cmap='gray', vmin=0, vmax=1.0)
# Plotting uses reversed y-axis now: larger y values are further down
plt.plot(GOAL[0], GOAL[1], 'ro', linewidth=5)
plt.plot([BOUNDARY_WEST, BOUNDARY_WEST, BOUNDARY_EAST, BOUNDARY_EAST, BOUNDARY_WEST],
        [BOUNDARY_NORTH, BOUNDARY_SOUTH, BOUNDARY_SOUTH, BOUNDARY_NORTH, BOUNDARY_NORTH],
        'b-')
plt.show()

# window around goal
plt.imshow(get_window(GOAL), cmap='gray', vmin=0, vmax=1.0)
plt.show()

plt.imshow(im[(BOUNDARY_NORTH-14):(BOUNDARY_SOUTH+14),(BOUNDARY_WEST-14):(BOUNDARY_EAST+14)], cmap='gray', vmin=0, vmax=1.0)
plt.show()

The following functions complete the definition of the environment. The agent's movements always go in the intended direction, but the distance travelled has a small random component.

Besides by reaching the goal, the episode also terminates after TIME_LIMIT (200) steps; at that point, the agent gets reward TIMEOUT_REWARD (-10).

In [ ]:
# Take an action in a state
# Returns: new state, reward (always -1)
def step(state, action):
    x, y = state
    if action == ACTION_NORTH:
        y -= AVG_MOVEMENT_SIZE + np.random.randint(5) - 2
        if y < BOUNDARY_NORTH:
            y = BOUNDARY_NORTH
    elif action == ACTION_SOUTH:
        y += AVG_MOVEMENT_SIZE + np.random.randint(5) - 2
        if y > BOUNDARY_SOUTH:
            y = BOUNDARY_SOUTH
    elif action == ACTION_WEST:
        x -= AVG_MOVEMENT_SIZE + np.random.randint(5) - 2
        if x < BOUNDARY_WEST:
            x = BOUNDARY_WEST
    elif action == ACTION_EAST:
        x += AVG_MOVEMENT_SIZE + np.random.randint(5) - 2
        if x > BOUNDARY_EAST:
            x = BOUNDARY_EAST
        
    reward = -1.0
    return (x, y), reward

# Is the state close enough to the goal to be considered a success?
# There is a margin for error, so that the agent can't jump over the goal
def is_goal_reached(state):
    if np.amax(np.abs(np.asarray(state) - np.asarray(GOAL))) <= AVG_MOVEMENT_SIZE / 2 + 1:
        return True
    else:
        return False
    
# Return a randomly chosen non-terminal state as starting state
def starting_state():
    while True:
        state = (np.random.randint(BOUNDARY_WEST, BOUNDARY_EAST + 1),
                 np.random.randint(BOUNDARY_NORTH, BOUNDARY_SOUTH + 1))
        if not is_goal_reached(state):
            return state

This class handles the function approximation, with several methods to query and update it. A neural network is used, similar to what can be used for MNIST digit recognition.

In [ ]:
class ValueFunction:
    def __init__(self, step_size):
        self.step_size = step_size

        # Use a small convolutional neural network to process the image
        # The network has four output nodes: one for each action
        # The final layer is initialized to zero, so that all value estimates are initially zero
        self.model = models.Sequential()
        self.model.add(layers.Conv2D(32, (5, 5), activation='relu',
                                     input_shape=(28, 28, 1), data_format='channels_last'))
        self.model.add(layers.MaxPooling2D((2, 2)))
        self.model.add(layers.Conv2D(64, (5, 5), activation='relu'))
        self.model.add(layers.MaxPooling2D((2, 2)))
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(64, activation='relu'))
        self.model.add(layers.Dense(4, kernel_initializer='zeros', activation='linear'))
        self.model.compile(loss='mean_squared_error', optimizer=optimizers.SGD(lr=step_size))

    # Return estimated action value of given state and action
    def value(self, state, action):
        if is_goal_reached(state):
            return 0.0
        # Get value estimates for this state combined with each of the four actions
        q = self.model.predict(get_window(state, reshape=True))
        if not np.all(np.isfinite(q)):
            print("* value() found nonfinite prediction:", q)
        # Return the value estimate for the requested (state, action) pair
        return q[0, action]

    # Return vector of estimated action values of given state, for each action
    def values(self, state):
        if is_goal_reached(state):
            return np.zeros(4)
        # Get value estimates for this state combined with each of the four actions
        q = self.model.predict(get_window(state, reshape=True))
        if not np.all(np.isfinite(q)):
            print("* values() found nonfinite prediction:", q)
        # Return the value estimate for the requested (state, action) pair
        return q[0, :]

    # learn with given state, action and target
    def learn(self, state, action, target):
        if not np.all(np.isfinite(target)):
            print("* learn() received nonfinite target:", target)
        # Get value estimates for this state combined with each of the four actions
        window = get_window(state, reshape=True)
        q = self.model.predict(window)
        if not np.all(np.isfinite(q)):
            print("* learn() found nonfinite prediction:", q)
        q[0, action] = target
        self.model.train_on_batch(window, q)

    # Return estimated state value, based on the estimated action values
    def state_value(self, state):
        return np.max(self.values(state))

# Plot the state value estimates. Use a larger stride for lower resolution.
def plot_state_values(value_function, stride=1):
    v = np.zeros(((BOUNDARY_SOUTH - BOUNDARY_NORTH + stride) // stride,
                  (BOUNDARY_EAST - BOUNDARY_WEST + stride) // stride))
    for i, y in enumerate(range(BOUNDARY_NORTH, BOUNDARY_SOUTH + 1, stride)):
        for j, x in enumerate(range(BOUNDARY_WEST, BOUNDARY_EAST + 1, stride)):
            v[i,j] = value_function.state_value((x, y))
    plt.imshow(v)
    plt.colorbar()
    plt.show()

Next comes your part. The following two functions are responsible for the agent's behavior. First, get_action should implement the epsilon-greedy policy, and return an action chosen according to that policy.

In [ ]:
# Choose action at state based on epsilon-greedy policy and valueFunction
import random

def get_action(state, value_function, epsilon):
    # Your code here
    draw = random.uniform(0, 1)
    
    if draw > epsilon:
    # greedy    
        values = value_function.value(state)
        choice = max(values)
        return choice
    
    else:
    #random    
        while(True):
            choice = random.uniform(0,4)
            if choice >= 0 and choice < 1:
                return ACTION_NORTH
            elif choice >= 1 and choice < 2:
                return ACTION_EAST
            elif choice >= 2 and choice < 3:
                return ACTION_SOUTH
            elif choice >=3 and choice < 4:
                return ACTION_WEST
        
    return ACTION_NORTH

The following function handles the interaction between agent and environment for a single episode. By passing the same value_function object in multiple calls to this function, the agent can learn from a sequence of such interactions.

Implement the missing parts of semi-gradient n-step Q-learning. This algorithm is almost exactly the same as the semi-gradient n-step Sarsa algorithm in section 10.2 of the book, except that the final term of $G$ should be computed as for Q-learning rather than as for Sarsa.

The main missing part is the computation of the update target. To be able to compute this, and to make the right call to value_function.learn, you'll also need to add code in other places in the function to keep track of previous states, actions and rewards. For full points, do this in a way that uses an amount of memory that depends on $n$, but not on the number of time steps the episode takes.

In [ ]:
# Semi-gradient n-step Q-learning with approximation
# valueFunction: action value function to learn
# n: number of steps for multi-step TD
# epsilon: parameter of epsilon-greedy policy                       
# learning: should the value function be updated during the simulation?
def semi_gradient_n_step_q_learning(value_function, n=1, epsilon=0.0, learning=True):
    current_state = starting_state()
    # get initial action
    current_action = get_action(current_state, value_function, epsilon)

    # track the time
    time = 0

    # termination time
    T = float('inf')
    
    total_reward = 0.0
    
    while True:
        if time < T:
            # take current action and go to the new state
            new_state, reward = step(current_state, current_action)

            if is_goal_reached(new_state):
                T = time + 1
            elif time >= TIME_LIMIT:
                T = time + 1
                reward = TIMEOUT_REWARD
            else:
                # choose new action
                new_action = get_action(new_state, value_function, epsilon)
            
            total_reward += reward

        # get the time index of the state to update
        update_time = time - n + 1
        if update_time >= 0 and learning:
            target = 0.0
            
            # Your code here
            
            # update the state value function
            #value_function.learn(?, ?, target) # Fill in the blanks
        if update_time == T - 1:
            break
        # go to next time step
        time += 1
        if time < T:
            current_state = new_state
            current_action = new_action

    return total_reward

Now it's time to test the performance of the algorithm on the environment. Plot estimates of the return as a function of the episode, for a few combinations of the parameters $n$ and $\alpha$. A reasonable starting point for $\alpha$ is 1e-5.

In [ ]:
# Your code here

Also use your code to estimate the return of an agent that picks all actions uniformly at random.

In [ ]:
# Your code here

Take the parameter values for which you saw the best performance. Are you satisfied with its performance? Use the plot_state_values function to get an idea of what is going on. Describe your observations in a few sentences.

In [ ]:
# Your code here

*Your answer here*

As a final note: In section 16.5 of the book, you'll find a description of the algorithm that learned to play ATARI games. At the core, it is the algorithm we used here, but with several additional techniques to improve its performance. Although we do not cover those techniques in this course or use them in this programming assignment, I hope this assignment has given you a better appreciation some of the issues that arise in reinforcement learning with function approximation.